# TVtoTV

Going to use the publicly available IMDB dataset for the tv shows. I have experience working with it from my movie to movie BFS solver, so hopefully the notebook work will be more or less trivial for this project.